### One can use these samples to enter when code asks for an input.

sample1 - The easiest way to earn points with Fetch Rewards is to just shop for the products you already love. If you have any participating brands on your receipt, you'll get points based on the cost of the products. You don't need to clip any coupons or scan individual barcodes. Just scan each grocery receipt after you shop and we'll find the savings for you.

sample2 = The easiest way to earn points with Fetch Rewards is to just shop for the items you already buy. If you have any eligible brands on your receipt, you will get points based on the total cost of the products. You do not need to cut out any coupons or scan individual UPCs. Just scan your receipt after you check out and we will find the savings for you.

sample3 = We are always looking for opportunities for you to earn more points, which is why we also give you a selection of Special Offers. These Special Offers are opportunities to earn bonus points on top of the regular points you earn every time you purchase a participating brand. No need to pre-select these offers, we'll give you the points whether or not you knew about the offer. We just think it is easier that way.

## Algorithm used to solve the problem.
### Term Frequency - Inverse Document Frequency
Term Frequency or TF is the frequency of the word occuring in the certain document.
Inverse Document Frequency or IDF is the inverse of frequency of the word occuring across all documents.
Finally we multiply TF and IDF of every word to get TFIDF value.

This algorithm is used for converting the text into numerics but also certain word are given more importance than others.
The IDF function returns higher values for words that occur rarely across all documents,
but the TF function returns higher values for wors that occur frequently in a given document.
Thus, the words occuring highly in one document but rarely across other documents are given higher values and vice-versa.

This algorithm thus helps us to get scores of the words in the documents that are useful to determine similarity between documents based on high impact words. Although we are using this algorithm for 2 documents, we can easily scale this algorithm for multiple documents as well.

### The metric used is cosine similarity.
Finally after getting the TFIDF for every word in the document, we essentially convert the document into an array of numerical values corresponding to the words.

The cosine similarity helps to get a score between 0 and 1 for similarity between these 2 arrays. Score of 1 means highly similar and score of 0 means highly unsimilar.

## Importing libraries

In [2]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import re

## The algorithm code block.

This has codes to take inputs from user, three functions that perform TF, IDF & TFIDF operations and a function to calculate cosine similarity for 2 1-D arrays.

In [7]:
# Taking the input from the user.

doc1 = str(input("Enter document 1: "))
doc2 = str(input("\nEnter document 2: "))

# Since grammatical correctness of the samples is unknown, I am considering only the words for comparison.

# This regular expression will remove all punctuation marks and only alphanumerics and _ and spaces would be retained.
doc1 = re.sub(r'[^\w\s]',"",doc1)
doc2 = re.sub(r'[^\w\s]',"",doc2)

#split so each word have their own string
doc1 = doc1.split(" ")
doc2= doc2.split(" ")

# I join them to get rid of duplicates. This will help us get the total vocabulary from the 2 entered documents.
vocabulary = set(doc1).union(set(doc2))

# Creating dictionaries to keep the count of occurence of every word within the document.
# Initially count of all words is zero.
CountWordDoc1 = dict.fromkeys(vocabulary, 0) 
CountWordDoc2 = dict.fromkeys(vocabulary, 0)

# Increasing the count if word exists in the document.
for word in doc1:
    CountWordDoc1[word]+=1
    
for word in doc2:
    CountWordDoc2[word]+=1




# TF is the term-frequency or the frequency of every word in the document.
# It is calculated as (Number of occurences in the document) / (Total number of words in the document)

def termFrequency( CountWordDoc, wordList):
    '''
    Args:
    
    CountWordDoc - The dictionary that keeps count of occurences ofevery word in the document.
    wordList - The list of words in the document; Input document after splitting.
    
    Returns:
    
    Dictionary with key value pair of word and its term-frequency.
    '''
    
    tf = {}
    totalWords = len(wordList)
    for word, count in CountWordDoc.items():
        tf[word] = count/float(totalWords)
    return tf

# Calculating the TF for every document
tfDoc1 = termFrequency(CountWordDoc1, doc1)
tfDoc2 = termFrequency(CountWordDoc2, doc2)




# IDF is the Inverse Document Frequency. That means, the inverse of occurence of the word across documents.
# It is calculated as (Total Number of Documents) / (Number of Documents in which the word occurs).
# A logarithm is taken to scale down the values.

def InverseDocFrequency(docList):
    '''
    Args:
    
    docList - List of dictionaries that keep the word count for every document.
    
    Return:
    
    A Dictionary that keeps track of idf for each word.
    '''
    
    N = len(docList)
    
    # Now word occurence is counted across all the documents.
    counts = [i+j for i,j in zip(list(docList[0].values()),list(docList[1].values()))]
    
    # A new dict is created to store these values for each word.
    idf = dict(zip(list(docList[0].keys()),counts))

    # Applying the idf formula for every word. +1 is done to avoid division by zero.
    for word, count in idf.items():
        idf[word] = 1 + (np.log((N+1)/(count+1)))
        
    return idf

# Calculate the idf.
idf = InverseDocFrequency([CountWordDoc1, CountWordDoc2])




# Multiplying TF and IDF of every word
# It is done as TF*IDF.

def TFIDF(tfDoc, idf):
    '''
    Args:
    
    tfDoc - Dictionary of Term Frequencies calculated for every word in the document.
    idf   - Dictionary of Inverse Document Frequencies calculated for all the words in the vocabulary.
    
    Returns:
    
    Dictionary containing Tf*IDF for every word in the document.
    '''
    
    tfidf = {}
    for word, val in tfDoc.items():
        tfidf[word] = val*idf[word]
    return tfidf

# Calculate the TF-IDF.
tfidfDoc1 = TFIDF(tfDoc1, idf)
tfidfDoc2 = TFIDF(tfDoc2, idf)




# Calculating the similarity using the cosine metrics.
def cosine_similarity(tfidfDocument1, tfidfDocument2):
    '''
    Args:
    
    tfidfDocument1 - dictionary of TFIDF values for words in the document 1.
    tfidfDocument2 - dictionary of TFIDF values for words in the document 2.
    
    Returns:
    
    Float value between 0 and 1 representing the cosine similarity between document1 and document2.
    '''
    
    a = list(tfidfDocument1.values())
    b = list(tfidfDocument2.values())

    # calculate Cosine Similarity
    # norm of list a is the square-root of sum of squares of all values in a.
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    
    return np.round(cos_sim,3)

print("\n\nCosine similarity between document 1 and document 2: ", cosine_similarity(tfidfDoc1,tfidfDoc2))
print("A score of 1 means highly similar and score of 0 means highly unsimilar.")

Enter document 1: The easiest way to earn points with Fetch Rewards is to just shop for the products you already love. If you have any participating brands on your receipt, you'll get points based on the cost of the products. You don't need to clip any coupons or scan individual barcodes. Just scan each grocery receipt after you shop and we'll find the savings for you.

Enter document 2: The easiest way to earn points with Fetch Rewards is to just shop for the items you already buy. If you have any eligible brands on your receipt, you will get points based on the total cost of the products. You do not need to cut out any coupons or scan individual UPCs. Just scan your receipt after you check out and we will find the savings for you.


Cosine similarity between document 1 and document 2:  0.613
A score of 1 means highly similar and score of 0 means highly unsimilar.
